In [213]:
from bokeh.io import output_notebook, push_notebook, show, export_png
from bokeh.models import Span, Label
from bokeh.layouts import row
from bokeh.plotting import figure
import localization as lx
import pandas as pd
from pathlib import Path

In [315]:
i_file = "../Data/20200502_Lab/UWB/uwb_20200502_2/uwb.csv"
r_file = "../Data/20200502_Lab/R/UWB_location_2.csv"
o_dir = "../Data/20200502_Lab/UWB/uwb_20200502_2/"
image_name = 'UWB座標.png'
csv_name = 'uwb_meas.csv'

In [316]:
anchor_list = {'An0014':[731.2, 914, 230.5], 'An0015': [548.4, 776.9, 238], 'An0030': [613, 914, 239], 'An0051': [731.2, 0, 183], 'An0054': [0, 0, 232.2]}
anchor_df = pd.DataFrame.from_dict(anchor_list, orient='index', columns=['x','y','z']).reset_index().rename(columns={"index": "Anchor"})

In [317]:
data = pd.read_csv(i_file)
r = pd.read_csv(r_file)
r['Duration'] = r['Duration'].round(2)
data['Duration'] = data['Duration'].round(2)

In [318]:
result = []
error = []
golden = []
duration = []
for name, group in data.groupby('Duration'):
    P=lx.Project(mode='3D',solver='LSE')
    t,label=P.add_target()
    group = group.reset_index(drop=True)
    for idx in range(0, len(group)):
        anchor_name = group['Anchor'][idx]
        anchor_xyz = tuple(anchor_df[['x','y','z']][anchor_df['Anchor'] == anchor_name].values[0])
        P.add_anchor(anchor_name,anchor_xyz)
        t.add_measure(anchor_name, group['Ranging'][idx])
    P.solve()
    result.append(t.loc.std())
    g_xyz = r[['x','y','z']][r['Duration'] == name].values[0]
    golden.append(g_xyz)
    e_x = abs(g_xyz[0] - t.loc.std()[0])
    e_y = abs(g_xyz[1] - t.loc.std()[1])
    e_z = abs(g_xyz[2] - t.loc.std()[2])
    error.append([e_x, e_y, e_z])
    duration.append(name)
g = pd.DataFrame(golden, columns=['x_real','y_real','z_real'])
m = pd.DataFrame(result, columns=['x_meas','y_meas','z_meas'])
e = pd.DataFrame(error, columns=['x_error','y_error','z_error'])
d = pd.DataFrame(duration, columns=['duration'])

LSE Geolocating...
LSE Geolocating...
LSE Geolocating...
LSE Geolocating...
LSE Geolocating...
LSE Geolocating...
LSE Geolocating...
LSE Geolocating...


In [319]:
df = pd.concat([g, m, e, d],axis=1)

In [320]:
output_notebook()
p1 = figure(
    title="UWB座標",
    x_axis_label="時間(s)",
    y_axis_label="座標(cm)",
    plot_width=600,
    plot_height=400,
)
p1.line(df.duration, df.y_real, legend_label="y軸實際值", line_width=2, color='red')
p1.line(df.duration, df.y_meas, legend_label="y軸UWB測量值", line_width=2, color='blue')
p1.legend.location = "top_left"

p2 = figure(
    title="UWB誤差",
    x_axis_label="時間(s)",
    y_axis_label="誤差(cm)",
    plot_width=600,
    plot_height=400,
)
p2.line(df.duration, df.y_error, legend_label="y軸誤差", line_width=2)
mean_ye = df.y_error.mean()
max_value = df.y_error.max()
max_index = df.y_error.idxmax()
p2.circle(x=df.duration[max_index], y=max_value,color='red', radius= 0.05)
max_label = Label(x=df.duration[max_index], y=max_value,
                 text='誤差最大值: %f' % (max_value),
                 x_offset=-185, y_offset=-10,
                 border_line_color='black',
                 background_fill_color='lightgray'
)
p2.add_layout(max_label)
vline = Span(
        location=mean_ye,
        dimension="width",
        line_color="red",
        line_dash="dashed",
        line_width=3,
)
p2.add_layout(vline)
mean_label = Label(x=df.duration[0], y=mean_ye,
                 text='平均誤差: %f' % (mean_ye),
                 x_offset=0, y_offset=5,
                 border_line_color='black',
                 background_fill_color='lightgray'
)
p2.add_layout(mean_label)
p2.legend.location = "top_left"
handle = show(row(p1, p2), notebook_handle=True)
push_notebook(handle=handle)

Loading BokehJS ...

In [321]:
Path(o_dir).mkdir(parents=True, exist_ok=True)
export_png(row(p1, p2), filename= o_dir + image_name)
df.to_csv(o_dir + csv_name ,index=False)